[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/rsinghlab/pyaging/blob/main/tutorials/tutorial_histonemarkchipseq.ipynb) [![Open In nbviewer](https://img.shields.io/badge/View%20in-nbviewer-orange)](https://nbviewer.jupyter.org/github/rsinghlab/pyaging/blob/main/tutorials/tutorial_histonemarkchipseq.ipynb)

# Bulk histone mark ChIP-Seq

This tutorial is a brief guide for the implementation of the seven histone-mark-specific clocks and the pan-histone-mark clock developed ourselves. Link to [preprint](https://www.biorxiv.org/content/10.1101/2023.08.21.554165v3).

We just need two packages for this tutorial.

In [1]:
import pandas as pd
import pyaging as pya

## Download and load example data

Let's download an example of H3K4me3 ChIP-Seq bigWig file from the ENCODE project.

In [2]:
pya.data.download_example_data('ENCFF386QWG')

|-----> 🏗️ Starting download_example_data function
|-----------> Downloading data to pyaging_data/ENCFF386QWG.bigWig
|-----------> in progress: 100.0000%
|-----> 🎉 Done! [194.4930s]


To exemplify that multiple bigWigs can be turned into a df object at once, let's just repeat the file path.

In [3]:
df = pya.pp.bigwig_to_df(['pyaging_data/ENCFF386QWG.bigWig', 'pyaging_data/ENCFF386QWG.bigWig'])

|-----> 🏗️ Starting bigwig_to_df function
|-----> ⚙️ Load Ensembl genome metadata started
|-----------> Downloading data to pyaging_data/Ensembl-105-EnsDb-for-Homo-sapiens-genes.csv
|-----------> in progress: 100.0000%
|-----> ✅ Load Ensembl genome metadata finished [3.0290s]
|-----> ⚙️ Processing bigWig files started
|-----------> Processing file: pyaging_data/ENCFF386QWG.bigWig
|-----------> in progress: 100.0000%
|-----------> Processing file: pyaging_data/ENCFF386QWG.bigWig
|-----------> in progress: 100.0000%
|-----> ✅ Processing bigWig files finished [10.8158s]
|-----> 🎉 Done! [25.8541s]


In [4]:
df.head()

,ENSG00000223972,ENSG00000227232,ENSG00000278267,ENSG00000243485,ENSG00000284332,ENSG00000237613,ENSG00000268020,ENSG00000240361,ENSG00000186092,ENSG00000238009,...,ENSG00000237801,ENSG00000237040,ENSG00000124333,ENSG00000228410,ENSG00000223484,ENSG00000124334,ENSG00000270726,ENSG00000185203,ENSG00000182484,ENSG00000227159
pyaging_data/ENCFF386QWG.bigWig,0.028616,0.030415,0.027783,0.028616,0.028616,0.028616,0.044171,0.036474,0.030784,0.03181,...,0.034435,0.006822,1.413119,0.029424,0.140005,0.049786,0.069296,0.332126,0.028596,0.028616
pyaging_data/ENCFF386QWG.bigWig,0.028616,0.030415,0.027783,0.028616,0.028616,0.028616,0.044171,0.036474,0.030784,0.03181,...,0.034435,0.006822,1.413119,0.029424,0.140005,0.049786,0.069296,0.332126,0.028596,0.028616


## Convert data to AnnData object

AnnData objects are highly flexible and are thus our preferred method of organizing data for age prediction.

In [5]:
adata = pya.preprocess.df_to_adata(df)

|-----> 🏗️ Starting df_to_adata function
|-----> ⚙️ Create anndata object started
|-----> ✅ Create anndata object finished [0.0029s]
|-----> ⚙️ Add metadata to anndata started
|-----------? No metadata provided. Leaving adata.obs empty
|-----> ⚠️ Add metadata to anndata finished [0.0004s]
|-----> ⚙️ Log data statistics started
|-----------> There are 2 observations
|-----------> There are 62241 features
|-----------> Total missing values: 0
|-----------> Percentage of missing values: 0.00%
|-----> ✅ Log data statistics finished [0.0010s]
|-----> ⚙️ Impute missing values started
|-----------> No missing values found. No imputation necessary
|-----> ✅ Impute missing values finished [0.0009s]
|-----> ⚙️ Add unstructured data to anndata started
|-----> ✅ Add unstructured data to anndata finished [0.0002s]
|-----> 🎉 Done! [0.0073s]


/Users/lucascamillo/mambaforge/envs/brain/lib/python3.9/site-packages/anndata/_core/anndata.py:1897: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


Note that the original DataFrame is stored in `X_original` under layers. is This is what the `adata` object looks like:

In [6]:
adata

AnnData object with n_obs × n_vars = 2 × 62241
    var: 'percent_na'
    uns: 'imputer_strategy', 'data_type'
    layers: 'X_original'

## Predict age

We can either predict one clock at once or all at the same time. For convenience, let's simply input a few clocks of interest at once. The function is invariant to the capitalization of the clock name. 

In [7]:
adata = pya.pred.predict_age(adata, ['H3K4me3', 'H3K9me3', 'PanHistone'])

|-----> 🏗️ Starting predict_age function
|-----> ⚙️ Set PyTorch device started
|-----------> Using device: cpu
|-----> ✅ Set PyTorch device finished [0.0005s]
|-----> 🕒 Processing clock: H3K4me3
|-----------> ⚙️ Load clock started
|-----------------> Downloading data to pyaging_data/h3k4me3.pt
|-----------------> in progress: 100.0000%
|-----------> ✅ Load clock finished [1.1568s]
|-----------> ⚙️ Check features in adata started
|-----------------> All features are present in adata.var_names.
|-----------> ✅ Check features in adata finished [0.0022s]
|-----------> ⚙️ Filter features and extract data matrix started
|-----------> ✅ Filter features and extract data matrix finished [0.0032s]
|-----------> ⚙️ Convert numpy array to tensor started
|-----------> ✅ Convert numpy array to tensor finished [0.0010s]
|-----------> ⚙️ Initialize model started
|-----------> ✅ Initialize model finished [0.0025s]
|-----------> ⚙️ Predict ages with model started
|-----------> ✅ Predict ages with model 

In [8]:
adata.obs.head()

,h3k4me3,h3k9me3,panhistone
pyaging_data/ENCFF386QWG.bigWig,53.998566,44.322887,54.021847
pyaging_data/ENCFF386QWG.bigWig,53.998566,44.322887,54.021847


Having so much information printed can be overwhelming, particularly when running several clocks at once. In such cases, just set verbose to False.

In [9]:
pya.data.download_example_data('ENCFF386QWG', verbose=False)
df = pya.pp.bigwig_to_df(['pyaging_data/ENCFF386QWG.bigWig', 'pyaging_data/ENCFF386QWG.bigWig'], verbose=False)
adata = pya.preprocess.df_to_adata(df, verbose=False)
adata = pya.pred.predict_age(adata, ['H3K4me3', 'H3K9me3', 'PanHistone'], verbose=False)

/Users/lucascamillo/mambaforge/envs/brain/lib/python3.9/site-packages/anndata/_core/anndata.py:1897: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


In [10]:
adata.obs.head()

,h3k4me3,h3k9me3,panhistone
pyaging_data/ENCFF386QWG.bigWig,53.998566,44.322887,54.021847
pyaging_data/ENCFF386QWG.bigWig,53.998566,44.322887,54.021847


After age prediction, the clocks are added to `adata.obs`. Moreover, the percent of missing values for each clock and other metadata are included in `adata.uns`.

In [11]:
adata

AnnData object with n_obs × n_vars = 2 × 62241
    obs: 'h3k4me3', 'h3k9me3', 'panhistone'
    var: 'percent_na'
    uns: 'imputer_strategy', 'data_type', 'h3k4me3_percent_na', 'h3k4me3_metadata', 'h3k9me3_percent_na', 'h3k9me3_metadata', 'panhistone_percent_na', 'panhistone_metadata'
    layers: 'X_original'

## Get citation

The doi, citation, and some metadata are automatically added to the AnnData object under `adata.uns[CLOCKNAME_metadata]`.

In [12]:
adata.uns['h3k4me3_metadata']

{'species': 'Homo sapiens',
 'data_type': 'histone_mark',
 'year': 2023,
 'implementation_approved_by_author(s)': '✅',
 'preprocessing': None,
 'postprocessing': None,
 'citation': 'de Lima Camillo, Lucas Paulo, et al. "Histone mark age of human tissues and cells." bioRxiv (2023): 2023-08.',
 'doi': 'https://doi.org/10.1101/2023.08.21.554165',
 'notes': 'This is still a preprint, so the model might change'}